Bu çalışmamda makine öğrenmesine giriş dersi için veri analiz işlemi hazırlandı.

Endonezya'da biriken teyit sayısı 2 Mart 2020'den sonra katlanarak artıyor. Enfeksiyonun birçok nedeni var, ancak çoğu vakanın Endonezya'daki yabancı ile temasa geçmeleri veya yabancı ülkeleri ziyaret etmeleri gibi benzer nedenleri var. Endonezya, 29 Mayıs 2020 tarihine kadar koronavirüs uyarısını "Darurat Nasional"(Ulusal Acil Durum) 'a yükseltti. Veri setinin söylediği gibi Java adası daha fazla etkilenirken, Endonezya'nın başkenti Jakarta koronavirüs tarafından yüksek oranda enfekte oldu.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Kütüphaneler

In [ ]:
import datetime
import seaborn as sns
import operator
import matplotlib.pyplot as plt
from ipywidgets import interact
from IPython.display import Markdown as md
from datetime import date, timedelta, datetime
from numpy import sqrt
from pandas import read_csv, to_datetime
from prettytable import PrettyTable
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
df = read_csv('/kaggle/input/indonesia-coronavirus-cases/confirmed_acc.csv').iloc[39:]
df = df.reset_index()

last_date = to_datetime(df['date'].iloc[-1], format='%m/%d/%Y')
data = [
            ['4/11/2020', 3842],
            ['4/12/2020', 4241],
            ['4/13/2020', 4557],
            ['4/14/2020', 4839],
            ['4/15/2020', 5136],
            ['4/16/2020', 5527],
            ['4/17/2020', 5923],
            ['4/18/2020', 6248],
            ['4/19/2020', 6575],
            ['4/20/2020', 6760],
            ['4/21/2020', 7135],
            ['4/22/2020', 7418],
            ['4/23/2020', 7775],
            ['4/24/2020', 8211],
            ['4/25/2020', 8607],
            ['4/26/2020', 8882],
            ['4/27/2020', 9096],
            ['4/28/2020', 9511],
            ['4/29/2020', 9971]
        ]

for k,v in data:
    if datetime.strptime(k, "%m/%d/%Y") > last_date:
        df = df.append({'date' : k, 'cases' : v} ,ignore_index=True)

# update last_date
last_date = to_datetime(df['date'].iloc[-1], format='%m/%d/%Y')

In [ ]:
df['index'] = df.index
# tanggal= tarih, kasus= vaka sayısı
df.columns = ['indeks', 'tanggal', 'kasus']
df

In [ ]:
df.info()

Hastalığı onaylanmış insanların yaş dağılımı

In [ ]:
patient = pd.read_csv("../input/indonesia-coronavirus-cases/patient.csv")
plt.figure(figsize=(10,6))
sns.set_style("darkgrid")
plt.title("Hastalığı onaylanmış insanların yaş dağılımı")
sns.kdeplot(data=patient['age'], shade=True)

In [ ]:
patient.head()

In [ ]:
cases = pd.read_csv("../input/indonesia-coronavirus-cases/cases.csv")
cases.tail()

Yaş ve cinsiyete göre durum dağılımı

In [ ]:
sns.set_style("whitegrid")
sns.FacetGrid(patient, hue='gender', size = 10)\
.map(plt.scatter, 'current_state', 'age')\
.add_legend()
plt.title('Yaş ve cinsiyete göre durum dağılımı')
plt.show()

Her ildeki hasta sayısı

In [ ]:
plt.figure(figsize=(15,5))
plt.title('Her ildeki hasta sayısı')
patient.province.value_counts().plot.bar();

In [ ]:
import plotly.express as px
import plotly.offline as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

fig = px.pie( values=patient.groupby(['province']).size().values,names=patient.groupby(['province']).size().index)
fig.update_layout(
    font=dict(
        size=15,
        color="#242323"
    )
    )   
    
py.iplot(fig)

Her Hastanedeki Hasta Sayısı

In [ ]:
plt.figure(figsize=(15,5))
plt.title('Her Hastanedeki Hasta Sayısı')
patient.hospital.value_counts().plot.bar();

# POLINOMAL REGRESYON

In [ ]:
degree = 2

x = df['indeks'].values.reshape(df['indeks'].size,1)
y = df['kasus'].values.reshape(df['kasus'].size,1)
polynomial_features= PolynomialFeatures(degree=degree)
x_poly = polynomial_features.fit_transform(x)

model = LinearRegression()
model.fit(x_poly, y)
y_poly_pred = model.predict(x_poly)

rmse = sqrt(mean_squared_error(y,y_poly_pred))
r2 = r2_score(y,y_poly_pred)

In [ ]:
def forecast_days_after(n_days_after):
    # polinom regresyon modelinin tahminini döndürür
    model_pred = model.predict(polynomial_features.fit_transform([[n_days_after]]))
    return '{:d}'.format(int(round(model_pred[0][0])))

start_date = to_datetime(df['tanggal'].iloc[0], format='%m/%d/%Y')
last_date = to_datetime(df['tanggal'].iloc[-1], format='%m/%d/%Y')

def display_forecast_days_after(n_days_after):
    # polinom regresyon modelinin tahminini döndürür
    end_date = start_date + timedelta(days=n_days_after)
    model_pred = model.predict(polynomial_features.fit_transform([[n_days_after]]))
    return end_date.strftime("%d %B %Y"), '{:d}'.format(int(round(model_pred[0][0])))

def days_passed_timedelta():
    # bir timedelta nesnesinin sonucunu döndürür
    # başlangıç tarihinden bu yana kaç gün geçti (başlangıç_tarihi)
    last_date = to_datetime(df['tanggal'].iloc[-1], format='%m/%d/%Y')
    return last_date.date() - start_date.date() 

def date_days_after(n_days_after, date_format="%d %B %Y", **kwargs):
    tanggal = start_date + timedelta(days=n_days_after)
    return tanggal.date().strftime(date_format)

def display_tabel_prediksi(days_after):
    # tahmin tablosu
    days_passed = days_passed_timedelta().days
    tabel_prediksi = PrettyTable(['ke-', 'tanggal','prediksi'])
    for x in range(days_passed+1,days_after+1):
        tabel_prediksi.add_row([x, date_days_after(x, date_format="%d-%m-%Y"), forecast_days_after(x)])
    print(tabel_prediksi)
    
def display_tabel_realisasi(lookback):
    # tahmin tablosu
    days_passed = days_passed_timedelta().days
    tabel_realisasi = PrettyTable(['tanggal','pred', 'real', 'p-r'])
    for x in range(days_passed, days_passed-lookback+1, -1):
        pred_ = int(forecast_days_after(x))
        real_ = y[x][0]
        eval_ = pred_ - real_    
        tabel_realisasi.add_row([date_days_after(x, date_format="%d-%m-%Y"), pred_, real_, eval_])
    print(tabel_realisasi)
    
def display_equation(model):
    for coef in model.coef_:
        pass
    streq = "$y = " +str(model.intercept_[0])
    for i,c in enumerate(coef):
        j = len(coef)-i-1
        if abs(c) > c:
            sign = "-"
        else:
            sign = "+"
        if i > 1:
            streq += sign +str(abs(c)) + " \cdot x^{"+str(i)+"}"
        elif i == 1:
            streq += sign +str(abs(c)) + " \cdot x"

    streq =   streq + "$"
    return md(streq)

def display_widget_forecast_days_after():
    print('son gün verileridir ke-' + str(days_passed_timedelta().days) + \
          ', Tahmini ileri almak için kaydırıcıyı sağa kaydırın (sağ ok tuşunu kullanın) ->klavyede):')
    @interact(x=(days_passed_timedelta().days+1, 365,1))
    def h(x=days_passed_timedelta().days+1):
        tabel_prediksi = PrettyTable(['hari ke-x', 'tanggal','prediksi akumulasi'])
        tabel_prediksi.add_row([x, date_days_after(x), forecast_days_after(x)])
        print(tabel_prediksi)

In [ ]:
# tabel parameter
tabel_parameter = PrettyTable(['parameter', 'nilai'])
tabel_parameter.add_row(['polynomial\ndegree', degree])
tabel_parameter.add_row(['RMSE','{:.10}'.format(rmse)])
tabel_parameter.add_row(['R^2', '{:.10}'.format(r2)])

In [ ]:
plt.scatter(x, y, s=10)
sort_axis = operator.itemgetter(0)
sorted_zip = sorted(zip(x,y_poly_pred), key=sort_axis)
x, y_poly_pred = zip(*sorted_zip)
plt.title('Endonezya daki kumulatif kasus covid-19\n')
plt.xlabel('\n1 Mart 2020 dan sonrası')
plt.ylabel('pozitif vaka')

plt.plot(x, y_poly_pred, color='m')
plt.show()
print(tabel_parameter)
display_equation(model)

In [ ]:
display_tabel_prediksi(60)

In [ ]:
display_forecast_days_after(100)

In [ ]:
display_tabel_realisasi(21)